In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
import requests
from PIL import Image
import time
import datetime
import os
import pytz
from urllib.parse import urlparse
import json

In [ ]:
# Set timezone to Nairobi
nairobi_timezone = pytz.timezone("Africa/Nairobi")

# Image URLs
image_urls = [
    "https://tdcctv.data.one.gov.hk/AID01217.JPG",
    "https://tdcctv.data.one.gov.hk/AID01216.JPG",
    "https://tdcctv.data.one.gov.hk/AID01215.JPG",
    "https://tdcctv.data.one.gov.hk/AID01214.JPG",
    "https://tdcctv.data.one.gov.hk/AID01213.JPG",
    "https://tdcctv.data.one.gov.hk/AID01212.JPG",
    "https://tdcctv.data.one.gov.hk/AID01211.JPG",
    "https://tdcctv.data.one.gov.hk/AID01210.JPG",
    "https://tdcctv.data.one.gov.hk/AID01209.JPG"
]

# YOLO Model Path
model_path = "yolov8l.pt"

# Main loop to download images and run YOLO
while True:
    try:
        # Get current time in Nairobi
        current_time = datetime.datetime.now(tz=nairobi_timezone).strftime("%Y%m%d%H%M%S")
        
        # Define folder structure
        base_dir = "dataset"
        image_dir = os.path.join(base_dir, "images", current_time)
        os.makedirs(image_dir, exist_ok=True)
        
        print(f"📂 Created directory: {image_dir}")

        # Download images
        downloaded_files = []
        for image_url in image_urls:
            response = requests.get(image_url, stream=True)
            if response.status_code == 200:
                image_name = os.path.basename(urlparse(image_url).path)
                file_path = os.path.join(image_dir, image_name)
                with open(file_path, "wb") as file:
                    file.write(response.content)
                downloaded_files.append(file_path)
                print(f"✅ Downloaded: {file_path}")
            else:
                print(f"❌ Failed to download: {image_url}")

        if not downloaded_files:
            print("⚠️ No images downloaded. Skipping YOLO detection...")
        else:
            # Run YOLO segmentation
            print("🚀 Running YOLO segmentation...")
            os.system(f'yolo task=segment mode=predict model="{model_path}" conf=0.45 source="{image_dir}" save=true save_txt=true')

        print("⏳ Waiting 2 minutes before next cycle...")
        time.sleep(120)

    except Exception as e:
        print(f"❌ Error: {e}")
        break  # Exit loop on failure